# Torch

## Control Flow & Weight Sharing 

As an example of dynamic graphs and weight sharing, we implement a very strange model: a fully-connected ReLU network that on each forward pass chooses a random number between 1 and 4 and uses that many hidden layers, reusing the same weights multiple times to compute the innermost hidden layers.

For this model we can use normal Python flow control to implement the loop, and we can implement weight sharing among the innermost layers by simply reusing the same Module multiple times when defining the forward pass.

We can easily implement this model as a Module subclass:

In [1]:
# -*- coding: utf-8 -*-

In [3]:
import torch
import random

In [4]:
class DynamicNet(torch.nn.Module):
    
    def __init__(self, D_in, H, D_out):
        
        """
        Construct three nn.Linear instantces that will use in the forward pass.
        """
        
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        
        return y_pred

In [5]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.

N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above

model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

for t in range(500):

    # Forward pass: Compute predicted y by passing x to the model
    
    y_pred = model(x)

    
    # Compute and print loss
    
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    
    # Zero gradients, perform a backward pass, and update the weights.
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 77.5766830444336
199 18.83847427368164
299 1.1507999897003174
399 0.7201645374298096
499 1.2610197067260742


-- by HanaRo, 2020/09/10